In [13]:
# Building a ranking loss function
import random
import json
import math
import numpy as np
from constants import HUMAN_ANNOTATION_PATH

with open(HUMAN_ANNOTATION_PATH) as f:
    _human_dataset = json.load(f)

num_pts = len(_human_dataset['annotations'][0]['kpts_3d']) / 3

pairs = np.asarray([(i, j) for i in range(num_pts) for j in range(num_pts) if i != j])

BATCH_NUM_PAIRS = 64

pair_idxs = np.random.choice(len(pairs), BATCH_NUM_PAIRS)
ps = np.take(pairs, pair_idxs, axis=0)
truth = np.take(truth, ps)
r = np.sign(truth[:,0] - truth[:,1], dtype=float)

In [14]:


def loss_numpy(pred, ps, r):
    pred = np.take(pred, ps)

    #Just the r != 0. Probably faster.
    # return np.sum(np.log(1 + np.exp(r * np.subtract(pred[:,1], pred[:,0]))))

    # np piecewise function
    # return np.sum(np.piecewise(r, [r == 0, r != 0], [lambda r: np.square(pred[:,1] - pred[:,0]), lambda r: np.log(1 + np.exp(r * np.subtract(pred[:,1], pred[:,0])))]))    

    # Writeover r = 0 case
    result = np.log(1 + np.exp(r * (pred[:,1] - pred[:,0])))
    goods = (r == 0)
    diffs = np.square(pred[:,1] - pred[:,0])
    result[goods] = diffs[goods]
    return np.sum(result)

truth = _human_dataset['annotations'][0]['kpts_3d'][1::3]
truth = np.asarray(truth)
print loss_numpy(truth, ps, r)


0.129422066509


In [15]:
%time [loss_numpy(truth, ps, r) for _ in range(10000)]

CPU times: user 292 ms, sys: 5.82 ms, total: 298 ms
Wall time: 296 ms


[0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.12942206650860261,
 0.1294220

In [12]:
with session = 

10.000045398899218